In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE197977"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE197977"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE197977.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE197977.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE197977.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tumour immune contexture is a determinant of anti-CD19 CAR T-cell efficacy in large B cell lymphoma"
!Series_summary	"Axicabtagene ciloleucel (axi-cel), an anti-CD19 chimeric antigen receptor (CAR) T-cell therapy approved for treatment of relapsed/refractory large B-cell lymphoma (LBCL), has comparable efficacy across conventional LBCL markers. We analysed whether pre- and posttreatment  tumour immune contexture determines clinical outcomes for axi cel–treated patients in the ZUMA-1 pivotal study. Longitudinal evaluation of the tumour microenvironment (TME) uncovered dynamic patterns that occurred rapidly after axi-cel (within 2 weeks) in responders—pronounced enhancement of T- and myeloid cell signatures and diminution of B cell signature. Clinical response and overall survival associated with high CD8+ T-cell density (Immunoscore) and immune gene expression (Immunosign21) in TME pretreatment, which was paralleled by blood CAR T-cell levels postt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Any, Dict, Optional, Callable, List

# 1. Gene Expression Data Availability
# Based on the Series_summary and Series_overall_design, this appears to be a dataset studying Large B-cell Lymphoma 
# with gene expression data (immune contexture, TME analysis)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 & 2.2 For trait (Large B-cell Lymphoma)
# From the sample characteristics, "bestresponse" at index 2 indicates clinical outcomes
# which is relevant to the trait (lymphoma response to treatment)
trait_row = 2

def convert_trait(value):
    if pd.isna(value) or value == 'NA':
        return None
    # Extract the value after colon and trim whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert response to binary (favorable vs unfavorable)
    # CR (Complete Response) and PR (Partial Response) are considered favorable outcomes
    # SD (Stable Disease), PD (Progressive Disease), ND (Not Determined) are unfavorable
    if value in ['CR', 'PR']:
        return 1  # Favorable response
    elif value in ['SD', 'PD', 'ND']:
        return 0  # Unfavorable response
    else:
        return None

# For age: Not available in the sample characteristics
age_row = None
convert_age = None

# For gender: Not available in the sample characteristics
gender_row = None
convert_gender = None

# 3. Save Metadata
# trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering using validate_and_save_cohort_info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# As trait_row is not None, we proceed with clinical data extraction
if trait_row is not None:
    # For GEO datasets, clinical data is typically extracted from the sample characteristics
    # observed in the previous step's output rather than from a separate file
    
    # Create a dataframe from the Sample Characteristics Dictionary provided in the previous output
    sample_data = {
        0: ['visit: SCREENING', 'visit: BASELINE', 'visit: DAY7-14', 'visit: FCBWK4', 'visit: PROGFCB'],
        1: ['molecular subgroup: GCB', 'molecular subgroup: NA', 'molecular subgroup: ABC', 'molecular subgroup: UNCLASSIFIED'],
        2: ['bestresponse: PR', 'bestresponse: CR', 'bestresponse: SD', 'bestresponse: PD', 'bestresponse: ND'],
        3: ['baseline tumour burden (mm2): 8877', 'baseline tumour burden (mm2): 667', 'baseline tumour burden (mm2): 2441',
            'baseline tumour burden (mm2): 7700', 'baseline tumour burden (mm2): 4248', 'baseline tumour burden (mm2): 3916',
            'baseline tumour burden (mm2): 4205', 'baseline tumour burden (mm2): 5056', 'baseline tumour burden (mm2): 355',
            'baseline tumour burden (mm2): 7161', 'baseline tumour burden (mm2): NA', 'baseline tumour burden (mm2): 2794',
            'baseline tumour burden (mm2): 792', 'baseline tumour burden (mm2): 5456', 'baseline tumour burden (mm2): 39658',
            'baseline tumour burden (mm2): 1997', 'baseline tumour burden (mm2): 1133', 'baseline tumour burden (mm2): 2924',
            'baseline tumour burden (mm2): 6557', 'baseline tumour burden (mm2): 1600', 'baseline tumour burden (mm2): 4371',
            'baseline tumour burden (mm2): 2200', 'baseline tumour burden (mm2): 7952', 'baseline tumour burden (mm2): 6262',
            'baseline tumour burden (mm2): 14354', 'baseline tumour burden (mm2): 396', 'baseline tumour burden (mm2): 2069',
            'baseline tumour burden (mm2): 1022', 'baseline tumour burden (mm2): 8162', 'baseline tumour burden (mm2): 735'],
        4: ['worst grade of ne: 3', 'worst grade of ne: 1', 'worst grade of ne: 2', 'worst grade of ne: 0', 'worst grade of ne: 4']
    }
    
    # Convert to DataFrame format suitable for geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(sample_data, orient='index')
    
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Trait data not available, skipping clinical feature extraction")


Clinical data preview: {0: [1.0], 1: [1.0], 2: [0.0], 3: [0.0], 4: [0.0], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [nan], 12: [nan], 13: [nan], 14: [nan], 15: [nan], 16: [nan], 17: [nan], 18: [nan], 19: [nan], 20: [nan], 21: [nan], 22: [nan], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE197977.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Tumour immune contexture is a determinant of anti-CD19 CAR T-cell efficacy in large B cell lymphoma"
Line 1: !Series_geo_accession	"GSE197977"
Line 2: !Series_status	"Public on Sep 23 2022"
Line 3: !Series_submission_date	"Mar 05 2022"
Line 4: !Series_last_update_date	"Sep 25 2022"
Line 5: !Series_pubmed_id	"36038629"
Line 6: !Series_summary	"Axicabtagene ciloleucel (axi-cel), an anti-CD19 chimeric antigen receptor (CAR) T-cell therapy approved for treatment of relapsed/refractory large B-cell lymphoma (LBCL), has comparable efficacy across conventional LBCL markers. We analysed whether pre- and posttreatment  tumour immune contexture determines clinical outcomes for axi cel–treated patients in the ZUMA-1 pivotal study. Longitudinal evaluation of the tumour microenvironment (TME) uncovered dynamic patterns that occurred rapidly after axi-cel (within 2 weeks) in responders—pronounced enhancement of T- and myeloid cell signatures 

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the expression data
# Based on the preview, the identifiers appear to be numeric IDs (1, 2, 3, etc.) 
# rather than standard human gene symbols like TP53, BRCA1, etc.
# These are platform-specific probe IDs that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...
Successfully extracted gene annotation data with 54374 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'ORF': ['A2M', 'ABCB1', 'ABL1', 'ADA', 'AICDA'], 'Accession': ['NM_000014.4', 'NM_000927.3', 'NM_005157.3', 'NM_000022.2', 'NM_020661.1']}

Column names in gene annotation data:
['ID', 'ORF', 'Accession']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify mapping columns
# From the gene annotation preview, we can see:
# - 'ID' column contains numeric identifiers (1, 2, 3...) matching the gene expression data
# - 'ORF' column contains gene symbols (A2M, ABCB1, ABL1...)

prob_col = 'ID'  # Column with probe IDs matching gene expression data
gene_col = 'ORF'  # Column with gene symbols

print(f"Mapping from {prob_col} to {gene_col}")

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Generated mapping dataframe with {len(mapping_df)} entries")
print("Mapping preview:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level data to gene expression data
print("Converting probe data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Generated gene expression data with {len(gene_data)} genes and {gene_data.shape[1]} samples")

# Preview the gene data (first few genes)
print("Gene expression data preview:")
print(preview_df(gene_data))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping from ID to ORF
Generated mapping dataframe with 54374 entries
Mapping preview:
{'ID': ['1', '2', '3', '4', '5'], 'Gene': ['A2M', 'ABCB1', 'ABL1', 'ADA', 'AICDA']}
Converting probe data to gene expression data...
Generated gene expression data with 721 genes and 74 samples
Gene expression data preview:
{'GSM5935018': [1758.44, 249.72, 288.1, 626.42, 1224.42], 'GSM5935019': [1370.35, 236.35, 513.03, 308.31, 187.35], 'GSM5935020': [738.43, 167.63, 310.33, 1564.71, 196.91], 'GSM5935021': [1368.94, 654.96, 313.61, 252.45, 750.25], 'GSM5935022': [1473.55, 59.57, 282.08, 277.77, 114.84], 'GSM5935023': [1599.26, 204.32, 229.36, 1598.12, 31.3], 'GSM5935024': [1634.0, 120.19, 534.17, 1124.63, 148.81], 'GSM5935025': [1313.76, 0.0, 530.56, 931.18, 0.0], 'GSM5935026': [2905.59, 216.34, 396.87, 174.79, 7.88], 'GSM5935027': [2388.75, 226.64, 406.93, 488.05, 0.0], 'GSM5935028': [4471.62, 734.27, 232.63, 542.25, 67.67], 'GSM5935029': [854.05, 88.53, 230.08, 894.34, 492.49], 'GSM5935030': [1785.

Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE197977.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# Get the gene data from previous step
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Use the normalize_gene_symbols_in_index function to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(normalized_gene_data.index)} genes")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already saved in step 2
print("\nLoading previously saved clinical data...")
if os.path.exists(out_clinical_data_file):
    clinical_df = pd.read_csv(out_clinical_data_file)
    is_trait_available = True
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
else:
    print("Clinical data file not found. Attempting to recreate it.")
    # Re-extract clinical features using correct parameters from Step 2
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the correct parameters from Step 2
    trait_row = 0
    
    def convert_trait(value):
        if pd.isna(value) or not isinstance(value, str):
            return None
        
        # Extract value after colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "healthy control" in value.lower():
            return 0
        elif "large b-cell lymphoma" in value.lower():
            return 1
        else:
            return None
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Re-created clinical data saved to {out_clinical_data_file}")
    
    clinical_df = selected_clinical_df
    is_trait_available = True if trait_row is not None else False

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not clinical_df.empty:
        # Convert clinical_df to format expected by geo_link_clinical_genetic_data
        # If clinical_df is already in the right format, just use it directly
        if not isinstance(clinical_df.index, pd.Index) or clinical_df.index.name != 'Large_B-cell_Lymphoma':
            # If we loaded from CSV, we need to transpose and set the index
            # First check if the trait is in the columns
            if trait in clinical_df.columns:
                # Just use the dataframe directly - it's already in the right format
                pass
            else:
                # Convert to the right format: transpose dataframe
                clinical_df_transposed = clinical_df.set_index(clinical_df.columns[0]).T
                clinical_df = clinical_df_transposed

        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Normalizing gene symbols...


After normalization: 0 genes
Normalized gene data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE197977.csv

Loading previously saved clinical data...
Loaded clinical data with shape: (1, 30)

Linking clinical and genetic data...
Created linked data with 75 samples and 29 features

Handling missing values...
After handling missing values: 1 samples and 4 features

Evaluating feature bias...
Quartiles for 'Large_B-cell_Lymphoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Large_B-cell_Lymphoma' in this dataset is severely biased.

Trait bias determination: True
Final linked data shape: 1 samples and 4 features

Performing final validation...
Abnormality detected in the cohort: GSE197977. Preprocessing failed.

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset has severe bias in the trait distribution
